<a href="https://colab.research.google.com/github/bguzman012/MisProyectos-D/blob/master/RedNeuronalConTensorFlowKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![LOGOUPS.png](attachment:LOGOUPS.png)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# TAREA REDES NEURONALES


## <span style="color:BLUE">1. CARGA DE DATASET</span>

In [0]:
#!pip install tensorflow==2.0.0-beta0

In [2]:
#Esta guía explica el proceso para crear una arquitectura de redes neuronales, posteriormente explica el proceso de 
#optimización y fine tuning. Para ello, primeramente se define una red neuronal y se evalúa. Posteriormente, se ajustan
#los parámetros y se evalúa el modelo final.

#El dataset se llama Titanic, lo pueden encontrar en: 
#Dataset Kaggle: https://www.kaggle.com/c/titanic
#También lo adjunto en el directorio de esta guía

#Con este problema se intenta predecir si una persona sobrevivió o no sobrevivió, este tipo de problemas es aplicable a otros 
#problemas de sobrevivencia ante desastres.
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder

etiquetas = ["buying","maint","doors","persons","lug_boot","safety","y"]

df = pd.read_csv('/carsito.csv', names=etiquetas, header=None)
df.head()

#Aquí solo con la vista podemos descartar muchas variables que son las siguientes:
#Nombre: No nos sirve porque no aporta información al resultado
#Ticket: Tampoco nos aporta Información
#Cabin: Podría aportarnos información pero tiene valores NaN (perdidos)
#PassangerID: Es el indice, no aporta información relevante
#Parch: Lo eliminaremos para tener una peor rendimiento para saber como optimizar el modelo en la siguiente sección

#Entonces nuestros datos de entrada “X” tendrán los siguientes valores:
#Sex: Genero de la persona
#Age: Edad de la persona
#SibSp: la cantidad de hermanos o esposa
#Fare: El impuesto que pagaron para embarcarse
#Embarked: Es la clase en donde se embarco la persona

Using TensorFlow backend.


,buying,maint,doors,persons,lug_boot,safety,y
0,4,4,2,2,1,1,0
1,4,4,2,2,1,2,0
2,4,4,2,2,1,3,0
3,4,4,2,2,2,1,0
4,4,4,2,2,2,2,0


## <span style="color:BLUE">2. PREPARACION DE DATOS - LIMPIEZA</span>

In [3]:
df = df.dropna(subset=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'y'])
df.head()

,buying,maint,doors,persons,lug_boot,safety,y
0,4,4,2,2,1,1,0
1,4,4,2,2,1,2,0
2,4,4,2,2,1,3,0
3,4,4,2,2,2,1,0
4,4,4,2,2,2,2,0


In [4]:
#Seleccionamos las variables escogidas



Xsubset = df[['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']]
#Xsubset = df[['Pclass', 'Sex', 'Embarked']]
#Xsubset.fillna(0)

#para separar nuestra variable dependiente de la independiente, haremos lo siguiente:

y = df.y.values
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(encoded_y)
print(Xsubset)
print(y)
type(Xsubset)

      buying  maint  doors  persons  lug_boot  safety
0          4      4      2        2         1       1
1          4      4      2        2         1       2
2          4      4      2        2         1       3
3          4      4      2        2         2       1
4          4      4      2        2         2       2
...      ...    ...    ...      ...       ...     ...
1723       1      1      5        5         2       2
1724       1      1      5        5         2       3
1725       1      1      5        5         3       1
1726       1      1      5        5         3       2
1727       1      1      5        5         3       3

[1728 rows x 6 columns]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


pandas.core.frame.DataFrame

## <span style="color:BLUE">3. PREPARACION DE DATOS - PREPROCESAMIENTO</span>

Generalmente cuando entrenamos modelos de “Deep Learning” antes de pasar los datos, todos los valores numéricos hay que normalizarlos, porque si tenemos valores muy altos y muy bajos en la misma tabla, al pasar por la función de activación, dependiendo cual sea, no lanzará resultados óptimos, por lo que se necesita un escalado, para que los valores queden a la misma escala, usaremos el StandarScaler, que es para valores, que tienen mucha diferencia entre ambos.

Embarked, Sex, y PClass, son valores categóricos nominales, es decir, son valores que pertenecen a categorías, por ejemplo, PClass representa la clase en que estaban embarcados los pasajeros, sin embargo, los valores son nominales puesto que un pasajero de primera clase no representa mas que uno de segunda, a efectos prácticos, a nivel matemático, si lo tratamos como un numero no suma ni resta. Podemos tratarlo como una categoría, porque tenemos una cantidad fija de clases a lo largo de los datos. Lo mismo pasa con Embarked.

Por ser valores categóricos nominales no podemos dar las categorías directamente, tenemos que realizar el proceso de Coding es decir, pasar las categorías en formato (one_hot), que consiste en poner tantos ceros como categorías, y para representar un valor se coloca un uno en la posición del valor, ejemplo:

3 colores: azul, rojo, verde. la primera categoría corresponde a tantos ceros como tantas categorías que se tienen: 
azul [1,0,0]
verde es el tercero, por lo que para representar ese valor irá un uno en esa posición
verde = [0,0,1]

Documentación ColumnTransformer: 
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

Creamos un Pipeline (un formateador o canal) de pre procesamiento con sk_learn, usaremos la función make_column_transformer
Link para aprender más: https://scikit-learn.org/stable/modules/compose.html

Los transformadores generalmente se combinan con clasificadores, regresores u otros estimadores para construir un estimador compuesto. La herramienta más común es un Pipeline (tubería).

Otro enlace con ejemplo: https://medium.com/vickdata/easier-machine-learning-with-the-new-column-transformer-from-scikit-learn-c2268ea9564c

In [36]:
preprocesador1 = make_column_transformer(
    (StandardScaler(),['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']))

X = preprocesador1.fit_transform(Xsubset)
print(X.shape[1])
print(X.shape)

#print(X)

#print(preprocesador1)

cnamesDataset1 = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

print(cnamesDataset1)

DatasetPreprocesado = pd.DataFrame(data=X,columns=cnamesDataset1)
print(DatasetPreprocesado.head())

DatasetPreprocesado.to_csv("/"+"DatasetPreprocesado.csv", sep=";",index = False) #sep es el separado, por defector es ","

6
(1728, 6)
['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
     buying     maint     doors   persons  lug_boot    safety
0  1.341641  1.341641 -1.341641 -1.336306 -1.224745 -1.224745
1  1.341641  1.341641 -1.341641 -1.336306 -1.224745  0.000000
2  1.341641  1.341641 -1.341641 -1.336306 -1.224745  1.224745
3  1.341641  1.341641 -1.341641 -1.336306  0.000000 -1.224745
4  1.341641  1.341641 -1.341641 -1.336306  0.000000  0.000000


## <span style="color:BLUE">4. DIVISION EN TRAIN Y TEST</span>

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

#Ahora preparamos el perceptron. Importamos las neuronas simples y el modelo secuencial
#Modelo secuencial quiere decir que agregaremos capas y se conectarán de manera automática, 
#Dense es la librería de neuronas simples.
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import model_from_json
print('Librerías importadas')

Librerías importadas


In [8]:
X_train.shape[1]

6

## <span style="color:BLUE">5. ALMACENAMIENTO Y CARGA DE MODELOS DE REDES NEURONALES</span>

In [0]:
#FUNCIONES PARA GuARDAR Y CARGAR CUALQUIER MODELO

#Guardar pesos y la arquitectura de la red en un archivo 

def guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos):
    print("Guardando Red Neuronal en Archivo")  
    # serializar modelo a JSON

    # Guardar los Pesos (weights)
    model.save_weights('/'+ nombreArchivoPesos+'.h5')

    # Guardar la Arquitectura del modelo
    with open('/' + nombreArchivoModelo+'.json', 'w') as f:
        f.write(model.to_json())

    print("Red Neuronal Grabada en Archivo")   
    
def cargarRNN(nombreArchivoModelo,nombreArchivoPesos):
        
    # Cargar la Arquitectura desde el archivo JSON
    with open('/' + nombreArchivoModelo+'.json', 'r') as f:
        model = model_from_json(f.read())

    # Cargar Pesos (weights) en el nuevo modelo
    model.load_weights('/' + nombreArchivoPesos+'.h5')  

    print("Red Neuronal Cargada desde Archivo") 
    return model

## <span style="color:BLUE">6. DISEÑO DE RED NEURONAL DE PRUEBA Y EVALUACION</span> 

In [22]:
#Construcción del Modelo o Arquitectura de Redes Neoronales
model = Sequential()

#La primera capa Dense recibe el numero de variables, que es la segunda dimensión de la matriz X, esto es X_train.shape[1]
#La primera capa tiene 32 neuronas. La función de activación es la función rectificadora.
model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
#La segunda capa tiene 64 neuronas. La función de activación es la función rectificadora.
model.add(Dense(64, activation='relu'))
#La capa de salida tiene 1 neurona. La capa de salida debe tener la misma dimensión como de cantidad de salidas queremos,
#por ejemplo, en este caso la salida "Survived" solo requiere 0 y 1. Puesto que 0 o 1 ocupan solo un valor dentro de cada dato,
#entonces 1 neurona es suficiente. La función de activación es sigmoide para clasificación por probabilidad.
model.add(Dense(4, activation='sigmoid'))

#Como tenemos dos posibles salidas "0 o 1", vamos a escoger que el error lo trate como una clasificación binaria, 
#el optimizador será nuestra función derivada que nos ayudará a determinar hacia donde mover los pesos.

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method

#imprimir arquitectura de la red
model.summary()
#Entrenamiento: 

#Entrenaremos por 100 epochs, el batch_size es un argumento importante, porque representa cada cuántos datos va a actualizar
#los pesos. Este es el método del gradiente descendiente estocástico que hace el proceso más eficiente y preciso.
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)
score = model.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
score = model.evaluate(X_test, y_test, verbose=0)
print('Resultado en Test:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#mostrar pesos de la red
#print(model.get_weights())

#Guardar pesos y la arquitectura de la red en un archivo 

nombreArchivoModelo='arquitectura_prueba'
nombreArchivoPesos='pesos_prueba'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

#Cargar pesos y la arquitectura
model2=cargarRNN(nombreArchivoModelo,nombreArchivoPesos) 

model2.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Meth

score = model2.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model2.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 32)                224       
_________________________________________________________________
dense_14 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 260       
Total params: 2,596
Trainable params: 2,596
Non-trainable params: 0
_________________________________________________________________
Resultado en Train:
acc: 99.71%
Resultado en Test:
acc: 97.69%
Guardando Red Neuronal en Archivo
Red Neuronal Grabada en Archivo
Red Neuronal Cargada desde Archivo
Resultado en Train:
acc: 99.71%
Resultado en Test:
acc: 97.69%


## <span style="color:BLUE">7. PREDICCION - USO DEL MODELO DE PRUEBA </span> 

In [23]:

def predict(buying=0, maint=0, doors=0, persons=0, lug_boot=0, safety=0):
    cnames = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
    data = [[buying, maint, doors, persons, lug_boot, safety]]
    my_X = pd.DataFrame(data=data, columns=cnames)
    my_X = preprocesador1.transform(my_X)
    return model.predict_classes(my_X)

print('Predicción:',predict(buying=2, maint=1, doors=5, persons=4, lug_boot=3, safety=3))
print('Predicción:',predict(buying=1, maint=3, doors=5, persons=5, lug_boot=2, safety=1))

Predicción: [3]
Predicción: [0]


## <span style="color:BLUE">8. DISEÑO DE EXPERIMENTOS - OPTIMIZACION Y FINE TUNING </span> 

Para darnos mejor idea de cuanto es nuestro error, realizaremos un proceso llamado "cross_validation", este proceso entrena la cantidad de veces que definamos y devuelve la métrica indicada para todos los pasos, esto es mas que nada porque en cada epoch la pérdida suele variar, entonces esto nos dará la precisión calculando la media de todas estas precisiones.

El proceso de Optimizacion consiste en reducir el error. Es decir, buscamos que la precision sea más alta. Mientras mayor es el accuracy será mejor el modelo de red neuronal para este problema.

El proceso de "Fine Tunning" consiste en buscar posibles errores, y combinaciones de parámetros que puedan mejorar el modelo.
Este proceso consume mucha memoria RAM, por lo tanto es recomendable usar alguna nube con mejores recursos a los locales, como por ejemplo: Google Colab. 

Enlaces: 

https://colab.research.google.com/notebooks/welcome.ipynb#recent=true

https://colab.research.google.com/notebooks/

Estrategia de Optimización: 

1.Compilación, 

2.Densidad de las capas de neuronas, y 

3.Agregar una cantidad de Dropout.

Dropout: basicamente lo que hace es apagar neuronas al azar con el fin de que las neuronas no se vuelvan tan dependientes de
los datos, es decir que se entrenen mejor para evitar el overfitting, para ello importaremos nuestra capa de dropout.
La capa dropout recibe como parámetro un numero entre 0 y 1 que representa el porcentaje de neuronas que va a desactivar en 
esa capa, por el momento quedará en 0.1, luego optimizaremos ese valor.

Con el modelo "GridSearchCV" podremos optimizar todos los parámetros

In [25]:
#importamos el algoritmo cross validator, y un wrapper que permitirá usar modelos de keras con scikit learn
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
print('Librerías importadas')
print(y_train)

Librerías importadas
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


## <span style="color:BLUE">9. DISEÑO DE RED NEURONAL DE BASE Y EVALUACION</span> 

In [26]:
#Evaluación del modelo original: Esto puede tardar unos 30 segundos.
def build_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

#El modelo se pasa como parámetro
estimator = KerasClassifier(build_fn=build_model, epochs=100, batch_size=64) 
#cv es la cantidad de veces de entrenamiento del modelo
#n_jobs es para ocupar mas de un procesador. El parámetro -1 indica que queremos utilizar todos los procesadores disponibles
accuracies=cross_val_score(estimator, X_train, y_train, cv=10, n_jobs=-1)

mean_acc=accuracies.mean()
print('Precision media: ', mean_acc)

##Como resultado al proceso de entrenamiento, tenemos un accuracy en promedio aprox. de: 0.4971. Intentaremos mejorarlo.
#Simplemente con optimizer "adam", el resultado aprox. es: 0.798. Pruebenlo. Cambien optimizer='adadelta' a optimizer='adam'
model=build_model() 
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)
model.summary()
print('Resultado en Train:')
score = model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

nombreArchivoModelo='arquitectura_base'
nombreArchivoPesos='pesos_base'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

y = np.argmax(y, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

Precision media:  0.9797466456890106
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32)                224       
_________________________________________________________________
dense_17 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 260       
Total params: 2,596
Trainable params: 2,596
Non-trainable params: 0
_________________________________________________________________
Resultado en Train:
acc: 99.64%
Resultado en Test:
acc: 97.69%
Guardando Red Neuronal en Archivo
Red Neuronal Grabada en Archivo


## <span style="color:BLUE">10. OPTIMIZACION Y FINE TUNING - COMPILACION </span> 

In [27]:
#1. Compilación: Prueba de mejores parámetros batch_size, epochs y optimizer
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.


def build_model(optimizer):
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model


#parámetros que queremos probar, y sus valores 
#probaremos con batch_size, epochs, y optimizador, con el fin de encontrar la mejor combinación entre estos tres parámetros.
parameters = parameters = {'batch_size': [16,32],
             'epochs':[100,200],
             'optimizer': ['adadelta', 'rmsprop']}

estimator = KerasClassifier(build_fn=build_model, verbose=0)
#Ahora no le pasamos los parámetros al KerasClasifier, porque se los pasaremos a través de GridSearchCV
#el argumento verbose=0 es para que no muestre salida, si lo dejamos en cero, no mostrará la barra de progreso del entrenamiento
#GridSearchCV: recibe como parámetros nuestro modelo, nuestros parámetros, la medida sobre la que queremos comparar, y la 
#cantidad de veces que lo entrenará para sacar la media de accuracy.
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
#Un ejemplo de resultados es: {'batch_size': 16, 'epochs': 100, 'optimizer': 'rmsprop'}
#Esto indica que el optimizador "adadelta" no es adecuado. Y es que este optimizador NO sirve para este tipo de problemas.

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'batch_size': 32, 'epochs': 200, 'optimizer': 'rmsprop'}


## <span style="color:BLUE">11. OPTIMIZACION Y FINE TUNING - DENSIDAD DE LAS CAPAS DE NEURONAS </span> 

In [28]:
#2. Densidad de las capas de neuronas
#Notemos que se incluyen los mejores parámetros del paso de optimización anterior (batch_size, epochs y optimizer)
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(l1, l2):
    model = Sequential()
    model.add(Dense(l1, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'l1':[16,32,64,128,256],
                           'l2':[16,23,64,128,256]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=32, epochs=200)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

#Resultados: {'l1': 32, 'l2': 16}
#Los resultados indican que hubo un error en la red original, las capas van desde la más densa, a la menos densa.

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'l1': 256, 'l2': 128}


## <span style="color:BLUE">12. OPTIMIZACION Y FINE TUNING - PROCESO CON DROPOUTS </span> 

In [29]:
#3. Proceso con dropouts: apagar un porcentaje de neuronas al azar con el fin de que las neuronas no se vuelvan tan 
#dependientes de los datos.
##Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(d1, d2):
    model = Sequential()
    model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(d1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(d2))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'d1':[0.1,0.2,0.3],
                            'd2':[0.1,0.2,0.3]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=32, epochs=200)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

#Resultados: {'d1':0.2, 'd2':0.3}

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'d1': 0.1, 'd2': 0.1}


## <span style="color:BLUE">13. EVALUACION DE MODELO OPTIMIZADO </span> 

In [33]:
#Evaluación del Modelo Final

finalModel = Sequential()
#Finalmente, veamos como mejoró nuestro modelo, vamos a repetir el proceso de la validación cruzada.
#Finalmente, veamos como mejoró nuestro modelo, vamos a repetir el proceso de la validación cruzada.
#Esto puede probarse localmente. Ya con los mejores parámetros evaluamos la red neuronal. Puede tardar un minuto.
def build_model():
    model = Sequential()
    model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=200)
accuracies = cross_val_score(estimator, X_train, y_train, cv=10, n_jobs=-1)
mean_acc = accuracies.mean()
std_acc = accuracies.std()
print('accuracies: ')
print(accuracies)
print('Precisión media: ', mean_acc)
print('Desviación media: ',std_acc)
#Y el resultado es:
#Precision media aprox.: 0.8067
#Pasamos de un 50% precisión a un 80% de precisión, por lo que se recomienda: 
#hacer siempre el proceso de fine tunning, porque ayudará a crear modelos correctos en la mayoría de los casos.

y = df.y.values
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(encoded_y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

model=build_model() 
model.fit(X_train, y_train, epochs=200, batch_size=16, verbose=1)
model.summary()

score = model.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

nombreArchivoModelo='arquitectura_optimizada'
nombreArchivoPesos='pesos_optimizados'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

accuracies: 
[0.97841728 0.99280578 1.         1.         0.99275362 0.99275362
 1.         0.97826087 1.         1.        ]
Precisión media:  0.993499118089676
Desviación media:  0.008201009376735503
Epoch 1/200
1382/1382 [==============================] - 0s 167us/step - loss: 0.5490 - acc: 0.7728
Epoch 2/200
1382/1382 [==============================] - 0s 97us/step - loss: 0.2733 - acc: 0.8777
Epoch 3/200
1382/1382 [==============================] - 0s 102us/step - loss: 0.1919 - acc: 0.9233
Epoch 4/200
1382/1382 [==============================] - 0s 97us/step - loss: 0.1552 - acc: 0.9443
Epoch 5/200
1382/1382 [==============================] - 0s 96us/step - loss: 0.1326 - acc: 0.9522
Epoch 6/200
1382/1382 [==============================] - 0s 97us/step - loss: 0.1156 - acc: 0.9573
Epoch 7/200
1382/1382 [==============================] - 0s 95us/step - loss: 0.1041 - acc: 0.9602
Epoch 8/200
1382/1382 [==============================] - 0s 105us/step - loss: 0.1008 - acc: 0.9602
Epo

## <span style="color:BLUE">14. COMPARACION ENTRE MODELO BASE Y MODELO OPTIMIZADO </span> 

In [34]:
#Modelo Base
print('MODELO BASE')
nombreArchivoModelo='arquitectura_base'
nombreArchivoPesos='pesos_base'
Selectedmodel=cargarRNN(nombreArchivoModelo,nombreArchivoPesos) 

#Selectedmodel.summary()
Selectedmodel.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method
print('Resultado en Train:')
score = Selectedmodel.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (Selectedmodel.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = Selectedmodel.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (Selectedmodel.metrics_names[1], score[1]*100))

#Modelo optimizado
print('MODELO OPTIMIZADO')
nombreArchivoModelo='arquitectura_optimizada'
nombreArchivoPesos='pesos_optimizados'
Selectedmodel=cargarRNN(nombreArchivoModelo,nombreArchivoPesos)    

#Selectedmodel.summary()

Selectedmodel.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method
print('Resultado en Train:')
score = Selectedmodel.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (Selectedmodel.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = Selectedmodel.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (Selectedmodel.metrics_names[1], score[1]*100))

MODELO BASE
Red Neuronal Cargada desde Archivo
Resultado en Train:
acc: 99.64%
Resultado en Test:
acc: 97.69%
MODELO OPTIMIZADO
Red Neuronal Cargada desde Archivo
Resultado en Train:
acc: 100.00%
Resultado en Test:
acc: 99.42%


## <span style="color:BLUE">15. PREDICCION - USO DEL MODELO OPTIMIZADO </span> 

In [35]:
#Predicciones con nuevos datos


def predict(buying=0, maint=0, doors=0, persons=0, lug_boot=0, safety=0):
    cnames = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
    data = [[buying, maint, doors, persons, lug_boot, safety]]
    my_X = pd.DataFrame(data=data, columns=cnames)
    my_X = preprocesador1.transform(my_X)
    return Selectedmodel.predict_classes(my_X)

print('Predicción:',predict(buying=2, maint=1, doors=5, persons=4, lug_boot=3, safety=3))
print('Predicción:',predict(buying=1, maint=3, doors=5, persons=5, lug_boot=2, safety=1))


Predicción: [3]
Predicción: [0]
